In [37]:
#import tensorflow as tf
#from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [38]:
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [39]:
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [40]:
import numpy as np

class Perceptron:
    def __init__(self, learning_rate=0.01, epochs=1000, num_summators_layer1=1, num_summators_layer2=1):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.num_summators_layer1 = num_summators_layer1
        self.num_summators_layer2 = num_summators_layer2
        self.weights_layer1 = None
        self.weights_layer2 = None
        self.bias_layer1 = None
        self.bias_layer2 = None

    def activation_function(self, net_input):
    # Sigmoid activation
        return 1 / (1 + np.exp(-net_input))

    def fit(self, features, targets):
        n_examples, n_features = features.shape

        # Initialize weights and biases
        self.weights_layer1 = np.random.uniform(size=(n_features, self.num_summators_layer1), low=-0.5, high=0.5)
        self.weights_layer2 = np.random.uniform(size=(self.num_summators_layer1, self.num_summators_layer2), low=-0.5, high=0.5)
        self.bias_layer1 = np.random.uniform(size=(self.num_summators_layer1), low=-0.5, high=0.5)
        self.bias_layer2 = np.random.uniform(low=-0.5, high=0.5)

        for _ in range(self.epochs):
            for example_index, example_features in enumerate(features):
            # Layer 1
                net_input_layer1 = np.dot(example_features, self.weights_layer1) + self.bias_layer1
                output_layer1 = self.activation_function(net_input_layer1)

                # Layer 2
                net_input_layer2 = np.dot(output_layer1, self.weights_layer2) + self.bias_layer2
                y_predicted = self.activation_function(net_input_layer2)

                # Update weights and biases
                self._update_weights(example_features, output_layer1, net_input_layer1, y_predicted, targets[example_index])

    def _update_weights(self, example_features, output_layer1, net_input_layer1, y_predicted, y_actual):
        # Layer 2
        error_layer2 = y_actual - y_predicted
        weight_correction_layer2 = self.learning_rate * error_layer2 * y_predicted * (1 - y_predicted)
        self.weights_layer2 = self.weights_layer2 + np.dot(output_layer1.reshape(-1, 1), weight_correction_layer2.reshape(1, -1))
        self.bias_layer2 = self.bias_layer2 + weight_correction_layer2

        # Layer 1
        error_layer1 = np.dot(weight_correction_layer2, self.weights_layer2.T) * output_layer1 * (1 - output_layer1)
        weight_correction_layer1 = self.learning_rate * error_layer1
        self.weights_layer1 = self.weights_layer1 + np.dot(example_features.reshape(-1, 1), weight_correction_layer1.reshape(1, -1))
        self.bias_layer1 = self.bias_layer1 + weight_correction_layer1

    def predict(self, features):
        # Layer 1
        net_input_layer1 = np.dot(features, self.weights_layer1) + self.bias_layer1
        output_layer1 = self.activation_function(net_input_layer1)

        # Layer 2
        net_input_layer2 = np.dot(output_layer1, self.weights_layer2) + self.bias_layer2
        y_predicted = self.activation_function(net_input_layer2)

        return y_predicted
    
    def evaluate(self, X, y):
        y_pred = self.predict(X)
        return accuracy_score(y, np.where(y_pred >= 0.5, 1, 0))
        #return accuracy_score(y, y_pred)


In [41]:
class Perceptron_bad:
    def __init__(self, learning_rate=0.2, hidden_layer_size = 1, activation='sigmoid'):
        self.learning_rate = learning_rate
        self.hidden_layer_size = hidden_layer_size
        self.activation = activation
        self.weights1 = None
        self.weights2 = None
        self.bias1 = None
        self.bias2 = None
        self.weights3 = None
        self.bias3 = None

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def _heaviside(self, x):
        return np.where(x >= 0, 1, 0)
    
    #функция активации, выбираем
    def _activation_function(self, x):
        if self.activation == 'sigmoid':
            return self._sigmoid(x)
        elif self.activation == 'heaviside':
            return self._heaviside(x)
        else:
            raise ValueError("no such activation function")
            
    #каждый столбец в X это один из двух входных узлов
    #здесь метод градиентного спуска для апдейта весов
    def fit(self, X, y, epochs = 100):
        n_samples, n_features = X.shape
        n_outputs = len(np.unique(y))

        self.weights1 = np.random.randn(n_features, self.hidden_layer_size)
        self.bias1 = np.zeros((1, self.hidden_layer_size))
        self.weights2 = np.random.randn(self.hidden_layer_size, 1)
        self.bias2 = np.zeros((1, 1))
        self.weights3 = np.random.randn(1, n_outputs)
        self.bias3 = np.zeros((1, n_outputs))

        for _ in range(epochs):
            for idx, x_i in enumerate(X):
                z1 = np.dot(x_i, self.weights1) + self.bias1
                a1 = self._sigmoid(z1)
                z2 = np.dot(a1, self.weights2) + self.bias2
                a2 = self._sigmoid(z2)
                z3 = np.dot(a2, self.weights3) + self.bias3
                a3 = self._softmax(z3)

                d3 = a3 - np.eye(n_outputs)[y[idx]]
                dw3 = np.dot(a2.T, d3)
                db3 = d3
                d2 = np.dot(d3, self.weights3.T) * a2 * (1 - a2)
                dw2 = np.dot(a1.T, d2)
                db2 = d2
                d1 = np.dot(d2, self.weights2.T) * a1 * (1 - a1)
                dw1 = np.dot(x_i.reshape(-1, 1), d1)
                db1 = d1

                self.weights3 -= self.learning_rate * dw3
                self.bias3 -= self.learning_rate * db3
                self.weights2 -= self.learning_rate * dw2
                self.bias2 -= self.learning_rate * db2
                self.weights1 -= self.learning_rate * dw1
                self.bias1 -= self.learning_rate * db1

    def predict(self, X):
        z1 = np.dot(X, self.weights1) + self.bias1
        a1 = self._sigmoid(z1)
        z2 = np.dot(a1, self.weights2) + self.bias2
        a2 = self._sigmoid(z2)
        z3 = np.dot(a2, self.weights3) + self.bias3
        a3 = self._softmax(z3)
        return np.argmax(a3, axis=1)

    def _softmax(self, x):
        return np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)
        
    def evaluate(self, X, y):
        y_pred = self.predict(X)
        accuracy = accuracy_score(y_test, y_pred)
        return accuracy
        #return accuracy_score(y, np.where(y_pred >= 0.5, 1, 0))
    


In [42]:
# Загрузка датасета Iris
iris = datasets.load_iris()
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [43]:
X = iris.data
y = iris.target

In [44]:
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [45]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
perceptron = Perceptron(learning_rate=0.01, epochs = 500)
perceptron.fit(X_train, y_train)

In [57]:
perceptron1 = Perceptron(learning_rate = 0.2, epochs = 1000, num_summators_layer1=2)
perceptron1.fit(X_train, y_train)

# Make predictions
predictions = perceptron1.predict(X_test)


In [58]:
perceptron1.evaluate(X,y)

0.6666666666666666

In [62]:
log_regr = LogisticRegression()
log_regr.fit(X_train, y_train)
accuracy_logistic = accuracy_score(y_test, log_regr.predict(X_test))
print("accuracy логистической регрессии:", accuracy_logistic)

accuracy логистической регрессии: 1.0


In [63]:
svm = SVC(kernel = 'sigmoid', C=4.0, gamma=0.01)#svm = SVC(C=1.0) #svm = SVC(kernel = 'sigmoid', C=1.0)
svm.fit(X_train, y_train)
accuracy_svm = accuracy_score(y_test, svm.predict(X_test))
print("accuracy SVM-sigmoid:", accuracy_svm)

accuracy SVM-sigmoid: 0.8333333333333334


In [64]:
svm = SVC(C=1.0)
svm.fit(X_train, y_train)
accuracy_svm = accuracy_score(y_test, svm.predict(X_test))
print("accuracy SVM-linear:", accuracy_svm)

accuracy SVM-linear: 1.0
